In [38]:
import pandas as pd
import datetime
import time
import pickle
import numpy as np
from sqlalchemy import create_engine, inspect


In [39]:
database_path = "restaurants.sqlite"
conn_string = f"sqlite:///{database_path}"

# Create an engine that can talk to the database
engine = create_engine(conn_string)


In [40]:
 # get all tables
inspector_gadget = inspect(engine)
tables = inspector_gadget.get_table_names()
for table in tables:
    print(table)
    
    # get all columns in table
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column)
    print()

AlcoholType
{'name': 'index', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'id', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'description', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}

BYOBCorkageType
{'name': 'index', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'id', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'description', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}

BestNightsType
{'name': 'index', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'id', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'description', 'type': TEXT(), 'nul

In [68]:
city = '"Tempe"'
state = '"AZ"'
mexican_restaurant = False
query = f"""
        SELECT
              name,
              address,
              city,
              state,
              postal_code,
              review_count,
              stars
        FROM
            Restaurants
        WHERE
            city IN ({city})
            AND state IN ({state})
            And is_mexican_restaurant = {mexican_restaurant};

           
            """     
df = pd.read_sql(query, con=engine)
df.head()

,name,address,city,state,postal_code,review_count,stars
0,"""Wong's Place""","""1825 E Baseline Rd.""",Tempe,AZ,85283,149,4.0
1,"""Subway""","""130 E University Dr, Ste C""",Tempe,AZ,85281,5,3.5
2,"""Riazzi's Italian Garden""","""2700 S Mill Ave""",Tempe,AZ,85282,141,3.5
3,"""WisePies Pizza & Salad""","""521 S College Ave, Ste 110""",Tempe,AZ,85281,14,4.0
4,"""Silver Mine Subs""","""655 W Warner Rd""",Tempe,AZ,85284,14,3.5


In [84]:
dietary_restictions = '"na"'
meal_type = '"na"'
mexican_restaurant = False
restraunt_type = '"na"'
query2 = f"""
        SELECT
              r.name,
              r.address,
              r.city,
              r.state,
              r.postal_code,
              r.review_count,
              r.stars
        FROM
            Restaurants as r
        INNER JOIN 
            RestaurantAttributes as ra
            ON r.business_id = ra.business_id        
        WHERE
            city IN ({city})
            AND state IN ({state})
            AND ra.DietaryRestrictionsType = {dietary_restictions}
            AND ra.GoodForMealType = {meal_type}
            And is_mexican_restaurant = {mexican_restaurant}
            And RestaurantsType = {restraunt_type};
           
            """     
attribute_df = pd.read_sql(query2, con=engine)
attribute_df.head()

,name,address,city,state,postal_code,review_count,stars
0,"""Wong's Place""","""1825 E Baseline Rd.""",Tempe,AZ,85283,149,4.0
1,"""Subway""","""130 E University Dr, Ste C""",Tempe,AZ,85281,5,3.5
2,"""Riazzi's Italian Garden""","""2700 S Mill Ave""",Tempe,AZ,85282,141,3.5
3,"""WisePies Pizza & Salad""","""521 S College Ave, Ste 110""",Tempe,AZ,85281,14,4.0
4,"""Silver Mine Subs""","""655 W Warner Rd""",Tempe,AZ,85284,14,3.5


In [65]:
attribute_df.DietaryRestrictionsType.value_counts()

na                               13828
DietaryRestrictionsvegetarian       14
DietaryRestrictionsvegan             6
Name: DietaryRestrictionsType, dtype: int64

In [66]:
attribute_df.GoodForMealType.value_counts()

na                      13599
GoodForMealdinner         134
GoodForMeallunch           57
GoodForMealbrunch          25
GoodForMealbreakfast       24
GoodForMealdessert          8
GoodForMeallatenight        1
Name: GoodForMealType, dtype: int64

In [72]:
attribute_df.RestaurantsType.value_counts()

na                          11558
RestaurantsGoodForGroups      404
RestaurantsTakeOut            273
RestaurantsTableService       230
RestaurantsDelivery            69
RestaurantsReservations        29
Name: RestaurantsType, dtype: int64

In [4]:
query3 = f"""
        SELECT
              r.name,
              rev.text,
              rev.likes

        FROM
            Reviews as rev
        INNER JOIN 
            Restaurants as r
            ON rev.business_id = r.business_id
                    INNER JOIN 
                        RestaurantAttributes as ra
                                    ON r.business_id = ra.business_id
        WHERE
            city IN ({city})
            AND state IN ({state})
            AND ra.DietaryRestrictionsType = {dietary_restictions}
            AND ra.GoodForMealType = {meal_type}
            And is_mexican_restaurant = {mexican_restaurant}
            And RestaurantsType = {restraunt_type};
            """     
tips_df = pd.read_sql(query3, con=engine)
tips_df.head()

NameError: name 'city' is not defined

In [41]:
loc_query = f"""
                        SELECT
              r.city,
              r.state

        FROM
            Restaurants as r

                    """

loc_df = pd.read_sql(loc_query, con=engine)
loc_df.head()

,city,state
0,Houston,PA
1,Toronto,ON
2,Fort Mill,SC
3,Montreal,QC
4,Frazer,PA


In [13]:
city_count = loc_df.city.nunique()
city_count

382

In [14]:
cities_list= loc_df.city.unique()
cities

array(['Houston', 'Toronto', 'Fort Mill', 'Montreal', 'Frazer',
       'Pittsburgh', 'Las Vegas', 'Charlotte', 'Oakdale', 'Phoenix',
       'Gilbert', 'Champaign', 'Montréal', 'Scottsdale', 'Cleveland',
       'Stuttgart', 'MESA', 'Tega Cay', 'Mentor-on-the-Lake', 'Chandler',
       'Fitchburg', 'Mississauga', 'Edinburgh', 'Cave Creek',
       'Monroeville', 'Markham', 'Mesa', 'North York', 'Strongsville',
       'Westlake', 'Tempe', 'Ludwigsburg', 'Henderson', 'Stoughton',
       'Pickering', 'Fellbach', 'Oregon', 'Caledon Village', 'Tolleson',
       'Aurora', 'Matthews', 'Oakville', 'Sheffield Lake', 'Outremont',
       'Concord', 'Madison', 'Lakewood', 'North Olmsted', 'Sun City',
       'Olmsted Falls', 'Verona', 'Richmond Hill', 'Avondale', 'Copley',
       'Woodbridge', 'Beachwood', 'Glendale', 'Brossard', 'Scarborough',
       'Medina', 'Lasalle', 'Bethel Park', 'Middleton', 'Georgetown',
       'Davidson', 'Brampton', 'Aspinwall', 'Newmarket', 'Willoughby',
       'Anthem', 'S

In [42]:
states_count = loc_df.state.nunique()
states = loc_df.state.unique()
states_count

23

In [16]:
states = list(states)

In [17]:
mystring = ''

In [43]:
for x in states:
    if x != states[-1]:
        mystring += '"' + x + '", '
    else:
        mystring += '"' + x + '"'


In [44]:
mystring

'"PA", "ON", "SC", "QC", "NV", "NC", "AZ", "IL", "OH", "BW", "WI", "EDH", "MLN", "IN", "CHE", "ELN", "HLD", "NYK", "FIF", "WLN", "ESX", "NI", "C""PA", "ON", "SC", "QC", "NV", "NC", "AZ", "IL", "OH", "BW", "WI", "EDH", "MLN", "IN", "CHE", "ELN", "HLD", "NYK", "FIF", "WLN", "ESX", "NI", "C"'

In [23]:
states = mystring
DietaryRestrictionsType = '"DietaryRestrictionsvegetarian", "DietaryRestrictionsvegan", "na"'
GoodForMealType = '"GoodForMeallunch", "GoodForMealdinner","na", "GoodForMealbrunch", "GoodForMealbreakfast", "GoodForMealdessert", "GoodForMeallatenight"'
is_mexican_restaurant = False and True
RestaurantsType = '"RestaurantsGoodForGroups", "RestaurantsReservations", "RestaurantsTakeOut", "RestaurantsDelivery", "RestaurantsTableService", "na"'
query2 = f"""
                        SELECT
              r.name,
              r.address,
              r.city,
              r.state,
              r.postal_code,
              r.review_count,
              r.stars
        FROM
            Restaurants as r
        INNER JOIN 
            RestaurantAttributes as ra
            ON r.business_id = ra.business_id        
        WHERE

            ra.DietaryRestrictionsType in ({DietaryRestrictionsType})
            AND ra.GoodForMealType in ({GoodForMealType})
            And r.is_mexican_restaurant in ({is_mexican_restaurant})
            And ra.RestaurantsType in ({RestaurantsType})
            And r.state in ({states});

                    """


df = pd.read_sql(query2, con=engine)
df.head()

,name,address,city,state,postal_code,review_count,stars
0,"""East Coast Coffee""","""737 West Pike St""",Houston,PA,15342,3,4.5
1,"""Alize Catering""","""2459 Yonge St""",Toronto,ON,M4P 2H6,12,3.0
2,"""Toast Cafe""","""2429 Hwy 160 W""",Fort Mill,SC,29708,6,3.5
3,"""Le Bistro Balmoral""","""305 Rue Sainte-Catherine O""",Montreal,QC,H2X 2A1,8,3.0
4,"""Carrabba's Italian Grill""","""245 Lancaster Ave""",Frazer,PA,19355,25,3.5


In [ ]:
query2 = f"""
        SELECT
              r.name,
              r.address,
              r.city,
              r.state,
              r.postal_code,
              r.review_count,
              r.stars
        FROM
            Restaurants as r
        INNER JOIN 
            RestaurantAttributes as ra
            ON r.business_id = ra.business_id        
        WHERE
            city IN ({city})
            AND state IN ({state})
            AND ra.DietaryRestrictionsType = {dietary_restictions}
            AND ra.GoodForMealType = {meal_type}
            And is_mexican_restaurant = {mexican_restaurant}
            And RestaurantsType = {restraunt_type};
           
            """     
attribute_df = pd.read_sql(query2, con=engine)
attribute_df.head()

In [45]:
def citystates_df():
    
    loc_query = f"""
                            SELECT
                  r.city,
                  r.state

            FROM
                Restaurants as r

                        """
    #create a df of all cities and states in df
    loc_df = pd.read_sql(loc_query, con=engine)
    
    
    return loc_df
loc_df.head()

,city,state
0,Houston,PA
1,Toronto,ON
2,Fort Mill,SC
3,Montreal,QC
4,Frazer,PA


In [49]:
def listcities(loc_df):
    cities_list= loc_df.city.unique()
    cities_list = list(cities_list)
    return cities_list
print(listcities(loc_df))

['Houston', 'Toronto', 'Fort Mill', 'Montreal', 'Frazer', 'Pittsburgh', 'Las Vegas', 'Charlotte', 'Oakdale', 'Phoenix', 'Gilbert', 'Champaign', 'Montréal', 'Scottsdale', 'Cleveland', 'Stuttgart', 'MESA', 'Tega Cay', 'Mentor-on-the-Lake', 'Chandler', 'Fitchburg', 'Mississauga', 'Edinburgh', 'Cave Creek', 'Monroeville', 'Markham', 'Mesa', 'North York', 'Strongsville', 'Westlake', 'Tempe', 'Ludwigsburg', 'Henderson', 'Stoughton', 'Pickering', 'Fellbach', 'Oregon', 'Caledon Village', 'Tolleson', 'Aurora', 'Matthews', 'Oakville', 'Sheffield Lake', 'Outremont', 'Concord', 'Madison', 'Lakewood', 'North Olmsted', 'Sun City', 'Olmsted Falls', 'Verona', 'Richmond Hill', 'Avondale', 'Copley', 'Woodbridge', 'Beachwood', 'Glendale', 'Brossard', 'Scarborough', 'Medina', 'Lasalle', 'Bethel Park', 'Middleton', 'Georgetown', 'Davidson', 'Brampton', 'Aspinwall', 'Newmarket', 'Willoughby', 'Anthem', 'Sainte-Thérèse', 'York', 'Hampton Township', 'Avalon', 'McMurray', 'Sun Prairie', 'Laval', 'Mayfield Heig

In [50]:
def liststates(loc_df):
    #Create a list of states
    states_list = loc_df.state.unique()
    states_list = list(states_list)
    return states_list
print(liststates(loc_df))

['PA', 'ON', 'SC', 'QC', 'NV', 'NC', 'AZ', 'IL', 'OH', 'BW', 'WI', 'EDH', 'MLN', 'IN', 'CHE', 'ELN', 'HLD', 'NYK', 'FIF', 'WLN', 'ESX', 'NI', 'C']


In [54]:
def createstring (search_list):   
    #Create a string of all the states for an all options
    search_string = ''
    for x in search_list:
        if x != search_list[-1]:
            search_string += '"' + x + '", '
        else:
            search_string += '"' + x + '"'
    return search_string
print(createstring(liststates(loc_df)))

"PA", "ON", "SC", "QC", "NV", "NC", "AZ", "IL", "OH", "BW", "WI", "EDH", "MLN", "IN", "CHE", "ELN", "HLD", "NYK", "FIF", "WLN", "ESX", "NI", "C"


In [ ]:
search_string = ''
    for x in search_list:
        if x != search_list[-1]:
            search_string += '"' + x + '", '
        else:
            search_string += '"' + x + '"'